# Paikkatietoanalyysi

Tässä osassa tehdään paikkatietoanalyysi. Tutkimuskysymyksemme on seuraavanlainen: Miten osassa 2 käsittelemämme salamahavainnot jakautuvat kuntien välillä?

Analyysin päävaiheet ovat suurinpiirten seuraavanlaiset:
1. Muutetaan osasta 2 tutut salamahavainnot paikkatiedoksi
2. Lasketaan salamahavainnot kunnittain
3. Visualisoidaan tuloksia

Otetaan aluksi käyttöön tarvitut kirjastot:

In [ ]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# 1. Taulukosta kartalle

Aloitetaan muuttamalla taulukkomainen salamahavaintodata paikkatiedoksi.

## Harjoitus - DataFramen luonti

Lue datakansiossa oleva `lightnings.csv`-tiedosto pandas DataFrameksi.
1. Vinkki: tee ensin tiedostopolku
2. Vinkki: käytä `read_csv`-metodia

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
lightnings = pd.read_csv(Path("./data/lightnings.csv"))
lightnings.head()

## DataFramesta GeoDataFrameksi

Muutetaan lightnings-DataFrame nyt GeoDataFrameksi:

In [ ]:
geo_lightnings = gpd.GeoDataFrame(lightnings)
geo_lightnings.head()

In [ ]:
type(geo_lightnings)

Tyyppi muuttui, mutta tarvitsemme edelleen geometriasarakkeen. Luodaan seuraavaksi sellainen.

Latitude ja longitude -sarakkeet sisältävät salamahavaintojen sijaintitiedot. GeoPandasissa on valmiina funktio tällaisten xy-tyyppisten sijaintitietojen muuttamiseen geometrioiksi: `points_from_xy`:

In [ ]:
points = gpd.points_from_xy(
    x=geo_lightnings["longitude"],
    y=geo_lightnings["latitude"],
)
points

Voimme nyt asettaa lopputuloksen GeoDataFramen geometriaksi `set_geometry`-metodilla:

In [ ]:
geo_lightnings = geo_lightnings.set_geometry(points)

geo_lightnings.head()

In [ ]:
geo_lightnings.plot(markersize=0.01)

## Harjoitus - koordinaattijärjestelmä

Vaikka salamahavainnot saatiin jo visualisoitua, puuttuu niistä edelleen tieto koordinaattijärjestelmästä:

In [ ]:
geo_lightnings.crs == None

Projisoi salamahavainnot samaan koordinaattijärjestelmään, kuin kohta käytettävä kunta-aineisto (epsg=3067). Huomaa, että sijainnit ovat nyt asteita, eli emme voi suoraan asettaa ETRS89 / TM35FIN koordinaattijärjestelmää.

1. Aseta ensin `geo_lightnings`-GeoDataFrame wgs 84 koordinaattijärjestelmään (`set_crs`). Epsg-koodi on 4326.
2. Uudelleenprojisoi (`to_crs`) vasta kun koordinaattijärjestelmä on asetettu. Ylikirjoita `geo_lightnings`-muuttuja uudella epsg 3067 -versiolla.

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
geo_lightnings = geo_lightnings.set_crs(epsg=4326)
geo_lightnings.crs

In [ ]:
geo_lightnings = geo_lightnings.to_crs(epsg=3067)
geo_lightnings.crs.name

# 2. Salamahavainnot kunnittain

Lasketaan seuraavaksi salamahavaintojen määrät kunnittain.

Luetaan ensin kunta-aineisto geodataframeen:

In [ ]:
municipalities = gpd.read_file(Path("./data/kunnat.gpkg"))

## Harjoitus - tarkasta koordinaattijärjestelmät

Ennen kuin etenemme pidemmälle, tarkasta, että salamahavinnot (`lightnings`) ja kunnat (`municipalities`) ovat samassa koordinaattijärjestelmässä.

1. Vinkki: tässä kannattaa hyödyntää vertailuoperaattoria

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
if municipalities.crs == geo_lightnings.crs:
    print("crs matches")

## Liitos sijainnin perusteella

Nyt voimme edetä tekemään sijainteihin perustuvan liitoksen salamahavaintojen ja kuntien välillä. Tarkoituksena on, että saisimme jokaiseen salamaan tiedon siitä, minkä kunnan alueelle se iski.

Tämä kannattaa toteuttaa `sjoin` eli spatial join -metodilla. Yhdistetään siis salamahavaintoihin tieto kunnista, perusteena, että havainto on kunnan alueella ("within"). Liitoksen tyypiksi otetaan "left", sillä haluamme tuloksena vain salamahavainnot.

In [ ]:
lightnings_with_municipality = geo_lightnings.sjoin(
    municipalities,
    predicate="within",
    how="left",
)

Operaation lopputuloksena on uusi GeoDataFrame, jossa jokaisella rivillä (eli salamahavainnolla) on tieto kunnasta:

In [ ]:
lightnings_with_municipality

Rivejä on sama määrä kuin salamahavaintoja:

In [ ]:
len(lightnings_with_municipality) == len(lightnings)

## Kuntakohtaiset salamamäärät

Nyt voimme laskea kuntakohtaiset summat salamahavainnoille ryhmittelemällä havainnot kunnan nimen mukaan. `size`-metodilla saamme laskettua kunkin ryhmän rivien (eli havaintojen) summan.

In [ ]:
lightnings_with_municipality.groupby("nimi").size()

Haluamme lopulta liittää summat takaisin kunta-aineistoon (`municipalities`-GeoDataFrame). Liitos kannattaa tehdä kuntien nimien perusteella. Järkevää olisi muodostaa siis DataFrame, jossa olisi havaintomäärien lisäksi sarake kuntanimille liitosta varten.

Antamalla `group_by`-metodin parametrille `as_index` arvon `False`, aiheutamme sen, että kuntien nimiä ei käytetä indeksinä, vaan niille luodaan oma sarake. Samalla lopputulos on siis automaattisesti DataFrame.

In [ ]:
lightnings_with_municipality.groupby("nimi", as_index=False).size()

Käytetään siis vaihtoehdoista jälkimmäistä, ja tallennetaan lopputulos muuttujaan `lightning_counts`:

In [ ]:
lightning_counts = lightnings_with_municipality.groupby("nimi", as_index=False).size()
lightning_counts.head()

Nimetään vielä sarakkeet hieman selkeämmin. Tämä onnistuu esimerkiksi antamalla uudet sarakenimet listana:

In [ ]:
lightning_counts.columns = ["nimi", "lightning_count"]
lightning_counts.head()

Nyt voimme liittää salamahavaintojen kuntakohtaiset summat kunta-aineistoon kuntien nimen perusteella:

In [ ]:
municipalities = municipalities.merge(
    lightning_counts,
    left_on="nimi",
    right_on="nimi",
    how="left",
)
municipalities

## Tulosten tarkistus

Tarkistetaan analyysimme nopealla visualisoinnilla kuntakohtaisista havaintomääristä.

Tämä onnistuu antamalla `plot`-metodin `column`-parametrille arvoksi se sarakke, jonka arvot haluamme visualisoida kartalla.

In [ ]:
municipalities.plot(
    column="lightning_count"
)

Kartalla huomaamme, että analyysissämme on yksi ilmiselvä ongelma: Kunnat eivät ole samankokoisia, joten pinta-alaltaan suurimmat kunnat saavat jo kokonsa vuoksi suurimpia summia. 

## Harjoitus - salamahavainnot suhteutettuna pinta-alaan

Korjaa äsken huomattu ongelma laskemalla pinta-alaan suhteutetut havaintomäärät. Tavoitteena on saada tulokseski joka kunnalle salamoiden määrä per neliökilometri.
1. Laske kunnille pinta-alat neliökilometreinä (Vinkki: käytä `area`-attribuuttia ja jaa tuloksena saadut neliömetrit 1 000 000:lla).
2. Laske `municipalities`-GeoDataFramelle uusi sarake "lightnings_per_km2", joka saa arvokseen havaintojen määrän jaettuna kunnan pinta-alalla neliökilometreissä.

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
areas_km2 = municipalities.area / 1000000
areas_km2

In [ ]:
municipalities["lightnings_per_km2"] = municipalities["lightning_count"] / areas_km2
municipalities.head()

## Harjoitus - sarakkeen visualisointi

Visualisoi nyt pinta-alaan suhteutetut havaintomäärät (eli juuri luomasi "lightnings_per_km2" -sarake).

In [ ]:
# Kirjoita ratkaisu


## Ratkaisu

In [ ]:
municipalities.plot(column="lightnings_per_km2")

## Tulosten tallentaminen

Voimme tallentaa luomamme GeoDataFramen vaikkapa tiedostoon tai tietokantaan. Tallennetaan se data-kansioon uuteen geopackage-tiedostoon:

In [ ]:
output_path = Path("./data/municipalities_with_lightnings.gpkg")

municipalities.to_file(output_path)

# 3. Tulosten visualisointi

## Staattinen kartta

Olemme nyt tyytyväisiä analyysin tulokseen, mutta visualisointi jättää vielä toivomisen varaa. Tutustutaan siis `plot`-metodiin hieman tarkemmin.

Otetaan ensin mukaan legenda, kasvatetaan kuvan kokoa hieman, ja vaihdetaan värit:

In [ ]:
municipalities.plot(
    column="lightnings_per_km2",
    legend=True,     # Otetaan legenda käyttöön
    figsize=(6, 6),  # Määritellään kuvan (figure) koko
    cmap="copper",   # Valitaan väriramppi
)

Arvojen luokittelua voi vaihtaa `scheme`-parametrilla:

In [ ]:
municipalities.plot(
    column="lightnings_per_km2",
    legend=True,
    figsize=(6, 6),
    cmap="copper",
    scheme="quantiles",  # Vaihdetaan luokitteluksi kvantiilit
)

Jos haluamme eroon kartan kehyksistä, voimme laittaa ne pois päältä `plot`-metodin palauttaman matplotlib axes-olion kautta:

In [ ]:
ax = municipalities.plot(  # Tallennetaan axes-olio muuttujaan
    column="lightnings_per_km2",
    legend=True,
    figsize=(6, 6),
    cmap="copper",
    scheme="quantiles",
)

ax.set_axis_off()  # Kehys pois päältä

Alla vielä hieman edistyneempi esimerkki kommentoituna:

In [ ]:
ax = municipalities.plot(
    column="lightnings_per_km2",   # Visualisoitava sarake
    legend=True,                   # Legenda päälle
    figsize=(6, 6),                # Kuvan koko
    scheme="quantiles",            # Luokittelu
    cmap="cividis",                # Värit
    
    # nodata-rivien tyylit voidaan antaa sanakirjana parametrille missing_kwds
    missing_kwds={
        "facecolor": "black",      # Täyttöväri
        "label": "Ei havaintoja",  # Selite
        
    },
    # Legendan tyylittely samaan tapaan
    legend_kwds={
        "bbox_to_anchor": (0.25, 1),        # Legendan sijainti
        "title": "Salamahavaintoja / km2",  # Legendan otsikko
    }
)
ax.set_axis_off()        # Kehys pois
plt.tight_layout(pad=0)  # Asetellaan kuva minimoiden tyhjä tila

# Tallennetaan kuva tiedostoon
plt.savefig(Path("./static_map.png"), dpi=200)

Tämän notebookin kanssa samasta hakemistosta pitäisi nyt löytyä kuva `static_map.png`.

## Interaktiivinen kartta

Tehdään vielä `explore`-metodilla interaktiivinen versio kartastamme.

Luodaan ensin kartta muuttujaan `m`. `explore`-metodilla on osittain samat parametrit kuin `plot`-metodilla, mutta myös eroja on. Alla kommentoitu esimerkki:

In [ ]:
m = municipalities.explore(
    column="lightnings_per_km2",        # Sarake
    tiles="CartoDB Positron nolabels",  # Taustakartta
    cmap="cividis",                     # Värit
    scheme="quantiles",                 # Luokittelu

    # Tarkempia tyylittelyasetuksia annetaan sanakirjana style_kwds-parametrille
    style_kwds={
        "fillOpacity": 1,  # Asetetaan läpinäkyvyys pois päältä
    },
    # Legendan tyylittely
    legend_kwds={
        "colorbar": False,  # Käytetään väripalkin sijasta kategorista legendaa
        "caption": "Lightnings per km2",
    }
)

# Näytetään kartta m-muuttujan avulla
m

In [ ]:
type(m)

Tallennus sujuu `save`-metodilla:

In [ ]:
m.save(Path("./interactive_map.html"))

Nyt tämän notebookin kanssa samaan hakemistoon ilmestyi `interactive_map.html` tiedosto. Se on html-tiedosto, eli sen voi avata web-selaimella.